# Advent of Code 2022: Day 23
https://adventofcode.com/2022/day/23


## Part 1
Count the number of empty tiles after moving the elves for 10 rounds

### Get the data into a list of strings

In [1]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')
# Remove empty value at the bottom of the list.
data_list = data_list[:-1]

### Function to preprocess the data into a dictionary

In [2]:
def preprocess_data(data):
  
  # Setup dictionary
  # to return.
  output_dict = {}
  for i in range(len(data)):
    
    # Split each row
    # into a list.
    item_split = list(data[i])
    
    # If a position contains a '#'
    # add the position to the dictionary.
    for j in range(len(item_split)):
      if item_split[j] == '#':
        output_dict[(i,j)] = True
  return output_dict

### Function to count the number of empty tiles

In [3]:
def empty_tiles(elf_pos):
  
  # Get the minimum and maximum rows 
  # and columns which contain an elf.
  row_min,row_max = float('inf'),float('-inf')
  col_min,col_max = float('inf'),float('-inf')
  for pos in elf_pos:
      row_min = min(row_min, pos[0])
      row_max = max(row_max, pos[0])
      col_min = min(col_min, pos[1])
      col_max = max(col_max, pos[1])
  
  # Calculate the number of empty tiles.
  return (row_max-row_min+1)*(col_max-col_min+1)-len(elf_pos)

### Function to perform the first half of a round

In [4]:
def first_half(elf_pos, dir, directions, check_list):
  
  # Get the current 'first' 
  # direction to look and
  # add it back to the end 
  # of the list.
  dir_id = dir.pop(0)
  dir.append(dir_id)
  
  # Setup dictionary
  # of proposed positions.
  propose_elves = {}

  # Go through every elf.
  for elf in elf_pos:
    
    # Get a list of the 8 positions
    # surrounding an elf.
    elf_surrounds = [(elf[0]+dire[0], elf[1]+dire[1]) for dire in directions]
    
    # Check if any of the 8 positions
    # contain an elf. If none of the
    # positions contain an elf, the
    # current elf does not need to move.
    check_res = [0,0,0,0,0,0,0,0]
    for s in range(8):
      if elf_surrounds[s] in elf_pos:
        check_res[s] = 1
    if sum(check_res) == 0:
            continue
    
    # If an elf does need to move,
    # check in the four directions.
    for c in range(4):
      
      # Get the three positions to check
      # for the current direction. 
      check_ids = check_list[(c+dir_id)%4]
      
      # If all three are empty, create 
      # the proposed position.
      if sum([check_res[cid] for cid in check_ids]) == 0:
        pos_proposed = (elf[0]+directions[check_ids[0]][0],elf[1]+directions[check_ids[0]][1])
        
        # Make an entry in the dictionary, with
        # the proposed position as key and current
        # position in a list as value. If the 
        # proposed position is already in the 
        # dictionary, append the current position
        # to the list. 
        if pos_proposed in propose_elves:
          propose_elves[pos_proposed].append(elf)
        else:
          propose_elves[pos_proposed] = [elf]
        break
  return propose_elves, dir

### Function to perform the second half of a round

In [5]:
def second_half(elf_pos, propose_elves):
  
  # Go through the proposed positions.
  for key, value in propose_elves.items():
    
    # If only one elf proposed
    # to move to this position,
    # execute the move by adding
    # the proposed position to the
    # main dictionary and deleting
    # the previous position.
    if len(value) == 1:
      elf_pos[key] = True
      del elf_pos[value[0]]
  return elf_pos, len(propose_elves) == 0

### Function to execute multiple rounds

In [6]:
def execute_rounds(data, stop = 10):
  
  # Setup the dictionary of positions.
  elf_pos = preprocess_data(data)
  
  # List of directions, in the format of
  # row/column difference from the
  # current position. 
  directions = [(-1,0),(1,0),(0,-1),(0,1), # 0 north, 1 south, 2 west, 3 east
              (-1,1),(1,1),(1,-1),(-1,-1)] # 4 NE, 5 SE, 6 SW, 7 NW

  # List of positions to check for each
  # of the four cardinal directions. The
  # values correspond to indeces in the
  # directions list. 
  check_list = [(0,4,7),(1,5,6),(2,7,6),(3,4,5)]

  # List of the four
  # cardinal directions.
  dir = [0,1,2,3]

  # If the moving should stop after 
  # a certain number of rounds.
  if stop > 0:
    for round in range(stop):
      propose_elves, dir = first_half(elf_pos, dir, directions, check_list)
      elf_pos, _ = second_half(elf_pos, propose_elves)
    return empty_tiles(elf_pos)
  
  # Otherwise run until no
  # elves need to move.
  else:
    n_round = 0
    while True:
      n_round += 1
      propose_elves, dir = first_half(elf_pos, dir, directions, check_list)
      elf_pos, check = second_half(elf_pos, propose_elves)
      if check:
        break
    return n_round

In [7]:
execute_rounds(data_list)

3757

## Part 2
Find out which round no elf no longer needs to move.

In [8]:
execute_rounds(data_list, False)

918